# Demonstrating Open AI's Chat Completion API
What you will learn:
  - How to format prompts
  - How to use system and user role
  - How to enforce JSON output
  - Learn some nice practices how to specify the output format

## Installation and setup

In [ ]:
! pip install openai

In [ ]:
# short cut
API_KEY = "YOUR_API_KEY_HERE" # don't publish your API keys (they are connected to a credit card, but limited to 1$)

# A better way to protect it, is to set it in the environment
# import os
# API_KEY = os.getenv('MYOPENAIKEY')

In [ ]:
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI(api_key=API_KEY)

## Defining a chat completion function

In [ ]:
def run_chat_turn(system_message: str, user_prompt: str, response_format:str = "text", model:str="gpt-4o-mini") -> str:
    """
    Sends a chat turn to the OpenAI model and returns the assistant's response.
    
    Args:
        system_message (str): The system message providing context or instructions for the assistant.
        user_prompt (str): The user's input prompt.
        response_format (str): "text" or "json_object"
    
    Returns:
        str: The assistant's response.
    """
    response = client.chat.completions.create(
        model=model,  # your PCL 1 API keys can use gpt-4o (15 times more expensive) or gpt-4o-mini
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt}
        ],
        temperature=1,
        max_tokens=1000,
        top_p=1,
        response_format={
        "type": response_format
    },
    )
    return response.choices[0].message.content

### Testing the function

In [ ]:
system_message = "You are a helpful assistant."
user_prompt = "Explain the benefits of learning Python."

In [ ]:
assistant_response = run_chat_turn(system_message, user_prompt)
print(assistant_response)

In [ ]:
system_message = "You are an AI that lies to people in absurd ways to make them think by themselves!"
user_prompt = "Explain the benefits of learning Python."

In [ ]:
assistant_response = run_chat_turn(system_message, user_prompt)
print(assistant_response)

## A real relation mining use case
 - Let's solve a more relevant problem by machine reading and machine generation of structured output
 - Can LLMs solve a specific relation mining problem?

In [ ]:
system_message = "You are an expert in mountaineering."
user_prompt = """Here is a paragraph from the OCR-ized British Alpine Magazine from 1969.

PARAGRAPH:
"The ranks ot ' famous ' climbers can never be closed .
Hut what does tame mean to climbing ?
The true climber seeks his fulfilment in the limitations set to him personally by the forces of Nature .
Whether this fulfilment is found in walking amongst mountains or in conquering extreme routes , it is all the same in the end .
With luck , climbing will never become a competitive ' Sport ' , nor appear in the Olympic Games , 
although already two Olympic Gold Medals have been bestowed to climbers one in iy}2 to the brothers Schmid 
for their first ascent ot the Matterhorn North face and another in 10,^6 to Günther Oskar Dyhrenfurth for his success in 
his great Himalayan undertakings .
"


An entity recognition system recognized the following mountain:
 - Matterhorn


The entity recognition system also recognized the following persons:
 - Günther Oskar Dyrthenfurth

Now, analyze and report for each person in a JSON data structure of the following format: 
    
    { 
      "mountain": MOUNTAIN, 
      "relations":[ 
        {"person_name": PERSON_NAME1, "is_mountaineer": true/false/null , "climbed_mountain": true/false/null},
        {"person_name": PERSON_NAME2, "is_mountaineer": true/false/null , "climbed_mountain": true/false/null},
        ...
        ] 
    }

Use your background knowledge and the paragraph to decide whether the recognized person is a mountaineer (true) or not (false). 
In case you don't know, set the value to null.

For each mountaineer (person with is_mountaineer: true), decide whether there is textual evidence in the paragraph 
that they actually climbed this mountain. Use the value null if the evidence in the paragraph is inconclusive.

"""


In [ ]:
assistant_response = run_chat_turn(system_message, user_prompt,response_format="json_object",model="gpt-4o")
print(assistant_response)

In [ ]:
# structured data from AI
import json
output = json.loads(assistant_response)
mountain = output["mountain"]
for person in output["relations"]:
    print(person["person_name"], person["is_mountaineer"])
    

Look at the original article: http://www.alpinejournal.org.uk/Contents/Contents_1969_files/AJ%201969%2047-57%20Heckmair%20Climbers.pdf